# Importing Dependencies

### Instance Segmentation of Powder Particles and Satellites

This example is used to generate a visualization of an individual image


In [25]:
## regular module imports
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import pickle
import skimage.io
import sys

## detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
)
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.structures import BoxMode
#from detectron2.evaluation import coco_evaluation
from detectron2.data.datasets.coco import convert_to_coco_json
from detectron2.evaluation.coco_evaluation import instances_to_coco_json
from detectron2.utils.visualizer import GenericMask
import pycocotools.mask as mask_util
from skimage import measure
from imantics import Polygons, Mask
from PIL import ImageEnhance, Image



## Setting System Path

In [2]:
## ampis
root = '../../../'
sys.path.append(root)
from ampis import data_utils, visualize, export_anno
EXPERIMENT_NAME = 'satellite' # can be 'particles' or 'satellite'

### Methods to create a new directory, geometrically augment an image or photometrically augment an image


In [29]:
def create_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)
        print("Created Directory : ", d)
    else:
        print("Directory already existed : ", d)
    return dir

def flip_and_save(name, horizontally, vertically, in_dir, out_dir, save=True):
    new_name = name
    img_path = Path(in_dir, name +'.png')
    img = cv2.imread(str(img_path))
    if horizontally:
        new_name += 'X'
        img = cv2.flip(img, 1)
    if vertically:
        new_name += 'Y'
        img = cv2.flip(img, 0)
    new_img_path = Path(out_dir, new_name +'.png')
    if save:
        cv2.imwrite(str(new_img_path), img)
    return new_name
print('')

def color_and_save(name, transformation, in_dir, out_dir,):
    #transformation: 0-1 = darker, 1 = no change, 1+ = lighter
    im = Image.open(in_dir + '/' + name + '.png')
    enhancer = ImageEnhance.Brightness(im)
    factor = transformation
    im_output = enhancer.enhance(factor)
    name_change = name
    if factor < 1:
        name_change += 'd'
    elif factor > 1: 
        name_change += 'b'
    else:
        name_change += 's'
    im_output.save(out_dir +  '/' +name_change + '.png')

    

# Transformations
The following methods either transform an image geometrically by rotating or mirroring it, or photometrically by adjusting the brightness. These methods transform all images in a directory, see below if you would like to do a transformation on a single image. Depending on which block you run, a directory can either be geometrically or photometrically altered, or both. 

## Geometric Transformations Only
Simply establish the directory file path in "files" to find the images, and then establish the directory to save the new images to. 
Make sure there are only images in this directory, or the "geometric" or "photometric" folders. 


In [23]:
IMPORT_DIR = '250x'          #Location where images are currently stored
EXPORT_DIR = '250x_geo'      #Location where augmented images should be stored

files = os.listdir(IMPORT_DIR) #Loading in image names
create_dir(EXPORT_DIR)         #Creating directory for new images if not yet created
img_names = []
for f in files:
    if f.split('.')[0] != 'geometric' and f.split('.')[0] != 'photometric': #Seperates generated folders from images to transform
        img_names.append(f.split('.')[0]) #Removes .png formatting to just have a list of the image names
    
for f in img_names: #Loops through every image and transforms
    if f != '': #Error handling to make sure not empty
        print("transforming: " + f)
        flip_and_save(f, True, True, IMPORT_DIR, EXPORT_DIR, save=True) #Mirrors Across Horizontal and Vertical Axis
        flip_and_save(f, False, True,IMPORT_DIR, EXPORT_DIR, save=True ) #Mirrors Across Vertical Axis
        flip_and_save(f, True, False, IMPORT_DIR, EXPORT_DIR, save=True) #Mirrors Across Horizontal Axis
        flip_and_save(f, False, False, IMPORT_DIR, EXPORT_DIR, save=True) #Non-Augmented

Created Directory :  250x_geo
transforming: HP743_10S_250x
transforming: HP743_9S_250x
transforming: HP743_8S_250x
transforming: HP743_5S_250x
transforming: HP743_4S_250x
transforming: HP743_3S_250x
transforming: HP743_7S_250x
transforming: HP743_2S_250x
transforming: HP743_1S_250x
transforming: HP743_6S_250x
transforming: HP743_12S_250x
transforming: HP743_11S_250x


## Photometric Transformations Only
Simply establish the directory file path in "files" to find the images, and then establish the directory to save the new images to
To modify how bright or dark the images are being transformed, simply adjust the final number in color_and_save(). 1 is unchanged. Anything less than 1 is darker and anything greater than 1 is brighter

In [30]:
IMPORT_DIR = '250x'          #Location where images are currently stored
EXPORT_DIR = '250x_photo'    #Location where augmented images should be stored
MIN_BRIGHTNESS = 0.6         #Brightness of darkened image
MAX_BRIGHTNESS = 1.4         #Brightness of lightened image


files = os.listdir(IMPORT_DIR) #Loading in image names
create_dir(EXPORT_DIR)         #Creating directory for new images if not yet created
img_names = []
for f in files:
    if f.split('.')[0] != 'geometric' and f.split('.')[0] != 'photometric': #Seperates generated folders from images to transform
        img_names.append(f.split('.')[0]) #Removes .png formatting to just have a list of the image names

for f in img_names:
    if f != '':
        print("transforming: " + f)
        color_and_save(f, MIN_BRIGHTNESS, IMPORT_DIR, EXPORT_DIR)
        color_and_save(f, 1, IMPORT_DIR, EXPORT_DIR)
        color_and_save(f, MAX_BRIGHTNESS, IMPORT_DIR, EXPORT_DIR)

Directory already existed :  250x_photo
transforming: HP743_10S_250x
transforming: HP743_9S_250x
transforming: HP743_8S_250x
transforming: HP743_5S_250x
transforming: HP743_4S_250x
transforming: HP743_3S_250x
transforming: HP743_7S_250x
transforming: HP743_2S_250x
transforming: HP743_1S_250x
transforming: HP743_6S_250x
transforming: HP743_12S_250x
transforming: HP743_11S_250x


## Geometric and Photometric Transformations 


### Setting Constants
Set the import and export directory as well as the target brightness of darkened and brighteneds images

In [31]:
IMPORT_DIR = '250x'                #Location where images are currently stored
EXPORT_DIR = '250x_photo-geo'      #Location where augmented images should be stored
MIN_BRIGHTNESS = 0.6               #Brightness of darkened image
MAX_BRIGHTNESS = 1.4               #Brightness of lightened image

### Performing Transformations

In [42]:
files = os.listdir(IMPORT_DIR) #Loading in image names
create_dir(EXPORT_DIR)         #Creating directory for new images if not yet created
create_dir('temp')
temp_img_names = []
for f in files:
    if f.split('.')[0] != 'geometric' and f.split('.')[0] != 'photometric': #Seperates generated folders from images to transform
        temp_img_names.append(f.split('.')[0]) #Removes .png formatting to just have a list of the image names
    
for f in temp_img_names: #Loops through every image and transforms
    if f != '': #Error handling to make sure not empty
        print("transforming: " + f)
        flip_and_save(f, True, True, IMPORT_DIR, 'temp', save=True) #Mirrors Across Horizontal and Vertical Axis
        flip_and_save(f, False, True, IMPORT_DIR, 'temp', save=True) #Mirrors Across Vertical Axis
        flip_and_save(f, True, False, IMPORT_DIR, 'temp', save=True) #Mirrors Across Horizontal Axis
        flip_and_save(f, False, False, IMPORT_DIR, 'temp', save=True) #Non-Augmented


files = os.listdir('temp') #Loading in image names
img_names = []
for f in files:
    if f.split('.')[0] != 'geometric' and f.split('.')[0] != 'photometric': #Seperates generated folders from images to transform
        img_names.append(f.split('.')[0]) #Removes .png formatting to just have a list of the image names

for f in img_names:
    if f != '':
        print("transforming: " + f)
        color_and_save(f, 0.6, 'temp', EXPORT_DIR)
        color_and_save(f, 1, 'temp', EXPORT_DIR)
        color_and_save(f, 1.4, 'temp', EXPORT_DIR)
        
temp_dir = os.listdir('temp') #Identifies all temp images
for i in temp_dir:
    os.remove('temp/' + i) #removes all temp images
os.rmdir('temp') #removes temp dir

Created Directory :  250x_photo-geo
Created Directory :  temp
transforming: HP743_10S_250x
transforming: HP743_9S_250x
transforming: HP743_8S_250x
transforming: HP743_5S_250x
transforming: HP743_4S_250x
transforming: HP743_3S_250x
transforming: HP743_7S_250x
transforming: HP743_2S_250x
transforming: HP743_1S_250x
transforming: HP743_6S_250x
transforming: HP743_12S_250x
transforming: HP743_11S_250x
transforming: HP743_8S_250xX
transforming: HP743_7S_250xXY
transforming: HP743_2S_250xY
transforming: HP743_3S_250xXY
transforming: HP743_10S_250x
transforming: HP743_8S_250xXY
transforming: HP743_9S_250x
transforming: HP743_5S_250xY
transforming: HP743_10S_250xX
transforming: HP743_8S_250x
transforming: HP743_5S_250x
transforming: HP743_3S_250xY
transforming: HP743_9S_250xY
transforming: HP743_6S_250xY
transforming: HP743_5S_250xXY
transforming: HP743_4S_250xXY
transforming: HP743_4S_250x
transforming: HP743_9S_250xX
transforming: HP743_9S_250xXY
transforming: HP743_2S_250xX
transforming: HP